#sklearn 을 활용한 Logistic Regression

In [1]:
# 4가지의 feature 를 기반으로 자전거 대여 횟수 예측 
# 미리 count 를 확인한 결과 min = 0 , max = 977
# 조건 : 대여수가 500 이상이면,
# 가정 : 자전거를 더 준비해야한다.

# 그럼 이때 하루에 자전거 대여 500건이 이상인 경우는? 

In [11]:
import pandas as pd
import numpy as np

# 필요한 라이브러리
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 

#성능 지표 함수 로딩
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


#데이터 불러오기
url='https://codepresso-online-platform-public.s3.ap-northeast-2.amazonaws.com/learning-resourse/python-machine-learning-20210326/bike-demand.csv'
df_bike = pd.read_csv(url)


#독립변수 슬라이싱
x=df_bike.iloc[:,5:9]
print(x.head(5))

# 종속변수 데이터 정하기 
# 대여건 500 이상 -> 1, 그렇지 않으면 0
# 이때, 원래의 count 자료는 연속된 수치형 자료로 존재하기때문에 이진 분류를 위한 로지스틱 회귀 불가
# 즉, 새로운 이진형태를 가지는 데이터 셋을 생성 필수.

df_bike['y'] = 1
#df_bike 에 y라는 컬럼이 있으면 새로 생성하고 값은 1
#만약 이미 존재한다면 그 안의 값을 모두 1 로 변환

df_bike.loc[df_bike['count']< 500, 'y'] = 0
y = df_bike['y']
# negative 하도록 설정 



#정규화
scaler = StandardScaler()
scaler.fit(x)
result = scaler.transform(x)
datalist = pd.DataFrame(data = result, columns = x.columns)

#데이터 셋 분리
x_train,x_test,y_train,y_test = train_test_split(datalist,y,test_size=0.3 , random_state =12)

#객체 생성
lo = LogisticRegression()
lo.fit(x_train,y_train)
y_pred = lo.predict(x_test)

#평균 정확도 측정
train_score = lo.score(x_train,y_train)
test_score = lo.score(x_test, y_test)
print("train :{0:.3f}".format(train_score))
print("test:{0:3f}".format(test_score))

#오차 행렬 생성
error = confusion_matrix(y_test, y_pred) #실제 정답 모델과 예측 모델
print("error:", error)

# 성능지표 
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print("accuracy:{0:.3f},precision:{1:.3f},recall={2:.3f}".format(accuracy, precision,recall))


   temp   atemp  humidity  windspeed
0  9.84  14.395        81        0.0
1  9.02  13.635        80        0.0
2  9.02  13.635        80        0.0
3  9.84  14.395        75        0.0
4  9.84  14.395        75        0.0
train :0.927
test:0.924066
error: [[3018    0]
 [ 248    0]]
accuracy:0.924,precision:0.000,recall=0.000


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


정확도 : 0.92 나머지는 다 0

결과: 500 미만인 경우에 예측은 잘하지만,
      500 이상의 경우에 대해서는 예측 값이 좋지 않다


이유: 데이터 학습의 불균형 때문인데, negative 의 값의 양이 훨씬 많아서 학습이 많이 되었기 때문이다.